In [ ]:
# Test that it works somewhat
import h5py
from matplotlib.axes import Axes
from wildfire.data_types import *
from wildfire.data_utils import *
from wildfire.training_utils import *
from matplotlib.axes import Axes
import matplotlib.pyplot as plt

def plot_metrics(ax: Axes, run_dir: str, key: str, only_modis: bool, only_weight: bool = False):
    paths = sorted(glob(f"{run_dir}_*/validation_metrics.json"))
    config_paths = sorted(glob(f"{run_dir}_*/config.json"))
    files = [json_load(p) for p in paths]
    config = json_load(config_paths[0])
    if only_weight:
        return config["pos_weight"]
    run_id = os.path.basename(run_dir)
    sensor = "MOD14" if config["is_modis"] else "VNP14"
    label = f"w={int(config['pos_weight'])}"
    if config["is_modis"] != only_modis:
        return
    # Convert color string to RGBA
    runs = []
    for i, by_epoch in enumerate(files):
        vals = [m[key] for m in by_epoch]
        runs.append(vals)
    mean = np.mean(runs, axis=0)
    std = np.std(runs, axis=0)
    x = np.arange(len(mean))
    ax.plot(x, mean, label=label)
    ax.fill_between(x, mean - std, mean + std, alpha=0.1)  # Std shading



ids = list(range(53, 63))
for only_modis in [True, False]:
    fig, ax = plt.subplots()
    items = []
    for run_id in ids:
        run_dir = config.runs_path + f"/{run_id}"
        weight = plot_metrics(ax, run_dir, "0.5/iou", only_modis, only_weight=True)
        items.append((weight, run_id))
    items.sort()
    for _, run_id in items:
        run_dir = config.runs_path + f"/{run_id}"
        plot_metrics(ax, run_dir, "0.5/iou", only_modis)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("IoU [%]")
    sensor = "MOD14" if only_modis else "VNP14"
    ax.set_title(f"Validation IoU across 5 runs ({sensor})")
    ax.legend(loc='lower right')
    out_path = os.path.join(config.root_path, "figures", f"ablation_pos_weight_{sensor}.pdf")
    fig.savefig(out_path, dpi=150)
    plt.show()
    print(out_path)
